In [ ]:
!pip install transformers

In [ ]:
!pip install sacremoses

In [ ]:
!pip install sentencepiece

In [ ]:
!git clone https://github.com/amontgomerie/question_generator/

In [ ]:
import os
os.chdir("/content/")
!gdown '1KJ66LbUiPH-uLzuSMvJRTGjXrM-XnHCY'
!gdown '1KHj25NWCVCZu7ViHz8bUAKL990ge_Jd4'
file = "/content/context.txt"
ref_file = "/content/references.txt"
output_file_name = "/content/bert_evaluated_hypothesis.txt"

In [ ]:

%cd question_generator/
%load questiongenerator.py
from questiongenerator import QuestionGenerator
from questiongenerator import print_qa
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"
qg = QuestionGenerator()
# file = 'articles/indian_matchmaking.txt'
with open(file, 'r') as a:
    article = a.read()
qa_list = qg.generate(
    article, 
    num_questions=10, 
    answer_style='all'
)
print_qa(qa_list)

In [ ]:
qa_list = qg.generate(
    article, 
    num_questions=81, 
    answer_style='all'
)
print_qa(qa_list)

In [ ]:
for qa in qa_list:
  print(qa)
  break

In [ ]:
questions = [x['question']+'\n' for x in qa_list]
answers = [x['answer'] for x in qa_list]
print(len(questions))

In [ ]:
out = open(output_file_name, "w")
out.writelines(questions)
out.close()

In [ ]:

f3 = open(ref_file, "r")
references = []
blref = []
for line in f3:
#     print(line)
    blref.insert(0, line)
    references.insert(0, line.split())
print(len(references))
f3.close()

Now, we evaluate the generated questions using the BLEU and METEOR scores of the generated questions with respect to the reference questions. For this purpose we use the nltk library, the use of which is illustrated here.

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)
METEOR = nltk.translate.meteor_score.meteor_score([reference], hypothesis)
print(f"{line}METEOR : {METEOR}\n")

In [ ]:
f2 = open(output_file_name, "r")
i = 0
BLEUdict = {}
METEORdict = {}
for line in f2:
#     print(line)
    i = i + 1
    hypothesis = line
    BLEUscore = nltk.translate.bleu_score.sentence_bleu(blref, hypothesis)
    BLEUdict[line] = BLEUscore
    print(f"{i} : {line}BLEU4 : {BLEUscore}\n")
    METEORscore = nltk.translate.meteor_score.meteor_score(references, hypothesis.split())
    METEORdict[line] = METEORscore
    print(f"METEOR : {METEORscore}\n")
f2.close()

In [ ]:
sorted(BLEUdict.items(), key=lambda item: item[-1], reverse = True)

In [ ]:
sorted(METEORdict.items(), key=lambda item: item[1], reverse = True)

In [ ]:
print(sum(BLEUdict.values()) / len(BLEUdict.values()))
print(sum(METEORdict.values()) / len(METEORdict.values()))
print(sum(sorted(BLEUdict.values(),reverse = True)[0:12]) / 12)
print(sum(sorted(METEORdict.values(),reverse = True)[0:12]) / 12)